In [ ]:
pip install convokit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=76093a860c2e98d9b3add03c35663db7e7f1da3d3e74732cfd94f48d3f90ea1d
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1c33e0d5525b231


In [ ]:
import pandas as pd
import convokit

In [ ]:
# Load the Friends Corpus
corpus = convokit.Corpus(filename=convokit.download('friends-corpus'))

No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [ ]:
# Find Chandler's speaker object
chandler_speaker = None
for speaker in corpus.iter_speakers():
    print(speaker)
    if speaker.id == "Chandler Bing":
        chandler_speaker = speaker
        print(chandler_speaker.print_speaker_stats())
        break

if chandler_speaker is None:
    raise ValueError("Chandler Bing not found in the corpus")

Speaker(id: 'Monica Geller', vectors: [], meta: ConvoKitMeta({}))
Speaker(id: 'Joey Tribbiani', vectors: [], meta: ConvoKitMeta({}))
Speaker(id: 'Chandler Bing', vectors: [], meta: ConvoKitMeta({}))
Number of Utterances: 8568
Number of Conversations: 1513
None


In [ ]:
# Define Chandler's character ID in the dataset
character_name = "Chandler Bing"

# Extract conversations where Chandler is speaking
# chandler_conersations = []
# for conv in chandler_speaker.iter_conversations():
#     chandler_conversations.append({
#         "conversation_id": conv.id,
#         "utterance_id": utt.id,
#         "text": utt.text,
#         "reply_to": utt.reply_to,
#         "speaker": utt.speaker.name
#     })
def getUttReplyTo(utt):
    if utt.reply_to == None:
        return utt.speaker.id
    else:
        return corpus.get_utterance(utt.reply_to).speaker.id

chandler_utterances = []
for utt in chandler_speaker.iter_utterances():
      chandler_utterances.append({
          "utterance_id": utt.id,
          "text": utt.text,
          "reply_to": getUttReplyTo(utt),
          "speaker": utt.speaker.id
      })

print(chandler_utterances[0])


{'utterance_id': 's01_e01_c01_u003', 'text': 'All right Joey, be nice. So does he have a hump? A hump and a hairpiece?', 'reply_to': 'Joey Tribbiani', 'speaker': 'Chandler Bing'}


In [ ]:

# Convert to DataFrame for easier manipulation
chandler_df = pd.DataFrame(chandler_utterances)
print(chandler_df)
# Merge with previous utterance if `reply_to` is available
# This will create conversational pairs (prompt, response) for training
paired_data = []
for _, row in chandler_df.iterrows():
    if row['reply_to']:
        # Find the previous utterance by ID
        previous_utt = chandler_df[chandler_df['utterance_id'] == row['reply_to']]
        if not previous_utt.empty:
            paired_data.append({
                "prompt": previous_utt.iloc[0]['text'],
                "response": row['text']
            })

# Convert paired data to DataFrame and save
paired_df = pd.DataFrame(paired_data)

# Save the structured data to a CSV for training
paired_df.to_csv("chandler_dialogues.csv", index=False)
print("Data saved to chandler_dialogues.csv")


          utterance_id                                               text  \
0     s01_e01_c01_u003  All right Joey, be nice. So does he have a hum...   
1     s01_e01_c01_u008                          Sounds like a date to me.   
2     s01_e01_c01_u010  Alright, so I'm back in high school, I'm stand...   
3     s01_e01_c01_u012  Then I look down, and I realize there's a phon...   
4     s01_e01_c01_u014                                      That's right.   
...                ...                                                ...   
8563  s10_e18_c09_u011                                      Where's Ross?   
8564  s10_e18_c11_u006  Look around, you guys. This was your first hom...   
8565  s10_e18_c11_u013                                          Oh, okay.   
8566  s10_e18_c11_u017                            Oh, it's gonna be okay.   
8567  s10_e18_c11_u021                                       Sure. Where?   

             reply_to        speaker  
0      Joey Tribbiani  Chandler Bing

In [ ]:
for conv in chandler_speaker.iter_conversations():
    dialogue_sequence = []
    for utt in conv.iter_utterances():
        # Track the entire conversation flow, but tag Chandler’s lines
        if utt.speaker.id == chandler_speaker.id:
            dialogue_sequence.append(f"Chandler: {utt.text} \n")
        else:
            dialogue_sequence.append(f"{utt.speaker.id}: {utt.text}")

In [ ]:
chandler_utterances = []
for utt in chandler_speaker.iter_utterances():
    dialogue_sequence = []
    if utt.reply_to != None:
        replyto_utt = corpus.get_utterance(utt.reply_to)
        if replyto_utt.speaker.id != 'TRANSCRIPT_NOTE':
           # dialogue_sequence.append(f"{replyto_utt.speaker.id}: {replyto_utt.text};\n {utt.speaker.id}: {utt.text} \n")
            chandler_utterances.append({
                "text": utt.text,
                "reply_to_speaker": replyto_utt.speaker.id,
                "reply_to_text": replyto_utt.text
            })
            #if len(dialogue_sequence[0]) > 500:
            #    print(dialogue_sequence)

chandler_df = pd.DataFrame(chandler_utterances)

# Print the first few rows to verify
print(chandler_df.head())

# Save the structured data to a CSV for training
chandler_df.to_csv("chandler_dialogues.csv", index=False)
print("Data saved to chandler_dialogues.csv")

                                                text reply_to_speaker  \
0  All right Joey, be nice. So does he have a hum...   Joey Tribbiani   
1                          Sounds like a date to me.    Monica Geller   
2  Then I look down, and I realize there's a phon...            #ALL#   
3                                      That's right.   Joey Tribbiani   
4  All of a sudden, the phone starts to ring. Now...    Phoebe Buffay   

                                       reply_to_text  
0  C'mon, you're going out with the guy! There's ...  
1  Okay, everybody relax. This is not even a date...  
2                          Oh, yeah. Had that dream.  
3                                     Instead of...?  
4                                                No.  
Data saved to chandler_dialogues.csv


In [ ]:
# model dataset specific for phi3 now, but later conisder making it json so templates can be applied
import json

#system_message = {
#    "role": "system",
#    "content": "You are Joey Tribbiani from the TV show Friends. Respond to questions and engage in conversations in his signature style."
#}
conversations = []
for conv in chandler_speaker.iter_conversations():
    conversation = ""
    #dialogue.append({"role": "System", "text": "You are Chandler"})
    for utt in conv.iter_utterances():
        if utt.speaker.id == character_name:
            conversation += f"<|assistant|>{utt.text}\n"
        else:
        if utt.speaker.id != 'TRANSCRIPT_NOTE':

            conversation += f"{utt.speaker.id}: {utt.text}\n"
    conversations.append(conversation)

chandler_utterances = []
for utt in chandler_speaker.iter_utterances():
    dialogue_sequence = []
    if utt.reply_to != None:
        replyto_utt = corpus.get_utterance(utt.reply_to)
        if replyto_utt.speaker.id != 'TRANSCRIPT_NOTE':
           # dialogue_sequence.append(f"{replyto_utt.speaker.id}: {replyto_utt.text};\n {utt.speaker.id}: {utt.text} \n")
            chandler_utterances.append({
                "text": utt.text,
                "reply_to_speaker": replyto_utt.speaker.id,
                "reply_to_text": replyto_utt.text
            })
            #if len(dialogue_sequence[0]) > 500:
            #    print(dialogue_sequence)

chandler_df = pd.DataFrame(chandler_utterances)

# Print the first few rows to verify
print(chandler_df.head())

# Save the structured data to a CSV for training
chandler_df.to_csv("chandler_dialogues.csv", index=False)
print("Data saved to chandler_dialogues.csv")



from convokit import Corpus, download

# Load the Convokit Corpus
corpus = Corpus(filename=download("friends-corpus"))

# Define a system message for the assistant (e.g., Joey's persona)
system_message = {
    "role": "system",
    "content": "You are Joey Tribbiani from the TV show Friends. Respond to questions and engage in conversations in his signature style."
}

# Prepare the formatted data for JSON
formatted_data = []

for conv in corpus.iter_conversations():
    messages = [system_message]

    # Iterate through the utterances (turns) in the conversation
    for utt in conv.iter_utterances():
        # Assign "user" role for other speakers and "assistant" role for Joey
        role = "user" if utt.speaker.id != "Joey" else "assistant"
        messages.append({
            "role": role,
            "content": utt.text
        })

    # Add the conversation to the dataset
    formatted_data.append({"messages": messages})

# Save the formatted dataset to a JSON file
with open("formatted_conversations.json", "w") as f:
    json.dump(formatted_data, f, indent=4)

print(f"Saved {len(formatted_data)} conversations to 'formatted_conversations.json'")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
